In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install transformers==2.1.0
!pip install gensim==4.1.2
!pip3 install --user underthesea

# !pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from pyvi import ViTokenizer, ViPosTagger
import regex as re
from underthesea import word_tokenize
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Thống kê số lượng data theo nhãn
count = {}
for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    key = line.split()[0]
    count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

__label__thể_thao 2611
__label__âm_nhạc 2595
__label__nhịp_sống 2613
__label__công_nghệ 2595
__label__thời_sự 2607
__label__thế_giới 2602
__label__thời_trang 2596
__label__du_lịch 2593
__label__sống_trẻ 2602
__label__giáo_dục 2603
__label__kinh_doanh 2597
__label__pháp_luật 2592
__label__giải_trí 2604
__label__phim_ảnh 2596
__label__xe_360 2602
__label__ẩm_thực 2482
__label__xuất_bản 2599
__label__sức_khỏe 2589


In [ ]:
# Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 18
vocab = {}
label_vocab = {}
for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    words = line.split()
    # lưu ý từ đầu tiên là nhãn
    label = words[0]
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words[1:]:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

# count = {}
# for word in vocab:
#     if len(vocab[word]) == total_label:
#         count[word] = min([label_vocab[x][word] for x in label_vocab])
        
# sorted_count = sorted(count, key=count.get, reverse=True)
# for word in sorted_count[:100]:
#     print(word, count[word])
text = []
label = []

for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(label)
encode_label = label_encoder.transform(label)

In [ ]:
import pandas as pd
df = pd.DataFrame(text, columns=["text"])
df.to_csv('text.csv', index=False)

In [ ]:
rd = pd.read_csv('text.csv')
rd[:10]

,text
0,sanchez không dự trận khai màn nh anh cùng ars...
1,những sao việt hóa_thân thành phó nháy trong m...
2,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
import pandas as pd
df = pd.DataFrame(encode_label, columns=["label"])
df.to_csv('label.csv', index=False)

In [ ]:
lb = pd.read_csv('label.csv')
lb[:10]

,label
0,11
1,16
2,5
3,0
4,11
5,12
6,10
7,13
8,1
9,12


In [ ]:
a = pd.read_csv("label.csv")
b = pd.read_csv("text.csv")
df_merge_col = pd.merge(a, b,left_index=True, right_index=True, how='outer')
df_merge_col.to_csv("data.csv", index=False)

In [ ]:
c = pd.read_csv("data.csv")
c[:10]

,label,text
0,11,sanchez không dự trận khai màn nh anh cùng ars...
1,16,những sao việt hóa_thân thành phó nháy trong m...
2,5,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,0,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,11,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,12,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,10,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,13,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,1,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,12,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
df = c.copy()
df[:10]

,label,text
0,11,sanchez không dự trận khai màn nh anh cùng ars...
1,16,những sao việt hóa_thân thành phó nháy trong m...
2,5,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,0,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,11,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,12,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,10,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,13,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,1,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,12,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
df.shape

(46678, 2)

In [ ]:
import keras
import keras.utils
from keras import utils as np_utils
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import utils as np_utils
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
all_sentences = []


for document in df['text']:
  sentences = document.split("\n")
  all_sentences.extend(sentences)


In [ ]:
Y=keras.utils.np_utils.to_categorical(df['label'])  # one hot target as required by NN.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_sentences, Y, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 0.25 x 0.8 = 0.2


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vectorizer = TfidfVectorizer(max_features=100)
vectorizer = vectorizer.fit(X_train)
vectorizer = vectorizer.fit(X_val)

# df_train = vectorizer.transform(X_train)
# df_val = vectorizer.transform(X_val)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


maxlen = 700

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_train = pad_sequences(sequences_train, maxlen=maxlen)

sequences_val = tokenizer.texts_to_sequences(X_val)
sequences_val = pad_sequences(sequences_val, maxlen=maxlen)

In [ ]:
sequences_test = tokenizer.texts_to_sequences(X_test)
sequences_test = pad_sequences(sequences_test, maxlen=maxlen)

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout

vocab_size = len(tokenizer.word_index) + 1
embedding_size = 100

input_tfidf = Input(shape=(100,))
input_text = Input(shape=(maxlen,))

model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(Flatten())
model.add(Dense(18,activation='softmax',kernel_regularizer='l1'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 700, 100)          7764800   
                                                                 
 flatten_1 (Flatten)         (None, 70000)             0         
                                                                 
 dense_1 (Dense)             (None, 18)                1260018   
                                                                 
Total params: 9,024,818
Trainable params: 9,024,818
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow import keras
from keras import optimizers

In [ ]:
epochs=100
batch_size=64

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
model.fit(sequences_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(sequences_val,y_val),callbacks=[es, mc])

Epoch 1/100
523/526 [============================>.] - ETA: 0s - loss: 1.7749 - accuracy: 0.6836
Epoch 1: val_loss improved from inf to 1.35455, saving model to best_model.h5
526/526 [==============================] - 4s 6ms/step - loss: 1.7723 - accuracy: 0.6844 - val_loss: 1.3545 - val_accuracy: 0.7754
Epoch 2/100
524/526 [============================>.] - ETA: 0s - loss: 1.2257 - accuracy: 0.8198
Epoch 2: val_loss improved from 1.35455 to 1.28632, saving model to best_model.h5
526/526 [==============================] - 3s 5ms/step - loss: 1.2258 - accuracy: 0.8197 - val_loss: 1.2863 - val_accuracy: 0.7918
Epoch 3/100
521/526 [============================>.] - ETA: 0s - loss: 1.1454 - accuracy: 0.8396
Epoch 3: val_loss improved from 1.28632 to 1.24664, saving model to best_model.h5
526/526 [==============================] - 3s 5ms/step - loss: 1.1456 - accuracy: 0.8396 - val_loss: 1.2466 - val_accuracy: 0.8080
Epoch 4/100
522/526 [============================>.] - ETA: 0s - loss: 1.0

In [ ]:
modelx = keras.models.load_model('best_model.h5')

In [ ]:
y_pred = (modelx.predict(sequences_test)> 0.5).astype("float")


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89       268
           1       0.91      0.79      0.85       267
           2       0.85      0.65      0.73       270
           3       0.65      0.23      0.34       228
           4       0.85      0.73      0.78       270
           5       0.90      0.48      0.63       252
           6       0.79      0.75      0.77       250
           7       0.90      0.91      0.91       270
           8       0.59      0.53      0.56       249
           9       0.90      0.77      0.83       252
          10       0.90      0.84      0.87       258
          11       0.96      0.91      0.94       264
          12       0.96      0.46      0.62       241
          13       0.85      0.81      0.83       277
          14       0.97      0.90      0.93       262
          15       0.96      0.85      0.90       247
          16       0.87      0.74      0.80       290
          17       0.95    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
modelx.evaluate(sequences_test, y_test, batch_size=64)

73/73 [==============================] - 0s 2ms/step - loss: 1.2065 - accuracy: 0.8121


[1.2065497636795044, 0.8121250867843628]